3. для пользователей находим любимые категории
4. фильтруем то что порекомендовать по тому чтобы они были в любимых категориях

In [1]:
from lightfm import LightFM
from scipy.sparse import coo_matrix
import pandas as pd
import ml_metrics as metrics

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


# validate

In [2]:
user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')
user_hist_df['ts'] = pd.to_datetime(user_hist_df['ts'])
user_hist_df['month'] = user_hist_df['ts'].dt.month

test_1 = user_hist_df[user_hist_df['month'] == 6]
test_2 = user_hist_df[user_hist_df['month'] == 7]
train = user_hist_df[~user_hist_df['month'].isin([6, 7])]

In [3]:
correct_1 = test_1.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()
correct_2 = test_2.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()


In [4]:
train['event'] = 1

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
train = train.drop_duplicates(['user_id', 'movie_id', 'event'])

In [6]:
model = LightFM(no_components=10)

train_df = train[['user_id', 'movie_id', 'event']]

In [7]:
user_encode = {u: i for i, u in enumerate(train['user_id'].unique())}
item_encode = {u: i for i, u in enumerate(train['movie_id'].unique())}

In [8]:
user_decode = {v: k for k, v in user_encode.items()}
item_decode = {v: k for k, v in item_encode.items()}

In [9]:
n_users, n_items = len(user_encode), len(item_encode)

n_users, n_items

(2055, 4732)

In [10]:
train['user_id'] = train['user_id'].apply(lambda x: user_encode[x])
train['movie_id'] = train['movie_id'].apply(lambda x: item_encode[x])

In [11]:
train_coo = coo_matrix(
    (train['event'], (train['user_id'], train['movie_id'])), shape=(n_users, n_items))

In [12]:
movies_database = pd.read_csv('../data/raw/movies.csv')
new_movies = movies_database[(movies_database['year']>= 2010) & 
                             (movies_database['id'].isin(list(item_encode.keys())))]['id'].values

In [89]:
movies_database = pd.read_csv('../data/raw/movies.csv')
new_movies = movies_database[(movies_database['year']>= 2010) & 
                             (movies_database['id'].isin(list(item_encode.keys()))) &
                             (movies_database['imdb_rating']>= 6)]['id'].values

moveis_views = train[train['movie_id'].isin([item_encode[x] for x in new_movies])]['movie_id'].value_counts()

items_to_recom = moveis_views[moveis_views>100].index
print(len(items_to_recom))

preapre movie features and user prefs

In [91]:
movies_database = pd.read_csv('../data/raw/movies.csv')
movies_to_recom = [item_decode[x] for x in items_to_recom]
movies_database = movies_database[movies_database['id'].isin(movies_to_recom)]
item_features = movies_database[['id', 'genres']]
item_features['id'] = item_features['id'].apply(lambda x: item_encode[x])

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [92]:
item_features['genres'] = item_features['genres'].str.split(',')

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
item_features = item_features.dropna()

In [94]:
movie_genres = {}

for i in item_features.iterrows():
    movie_genres[i[1][0]] = i[1][1]

In [95]:
users_to_predict = [user_encode[x] for x in list(test_1['user_id'])]
items_to_predict = set(items_to_recom.copy())

In [96]:
movie_user_hist = train[train['episode_id'] == 0].groupby('user_id')['movie_id'].apply(set).to_dict()
user_hist = train.groupby('user_id')['movie_id'].apply(set).to_dict()
#user_hist = {user_encode[k]: v for k, v in user_hist.items()}

In [97]:
from collections import Counter

In [98]:
user_pref = {}
for user in users_to_predict:
    #print(user)
    user_pref[user] = {}
    #print(user_hist[user])
    for film in user_hist[user]:
        if movie_genres.get(film):
            for genre in movie_genres[film]:
                if user_pref[user].get(genre):
                    user_pref[user][genre] += 1
                else:
                    user_pref[user][genre] = 1
                    
    user_pref[user] = [key for key, val in Counter(user_pref[user]).most_common(15)]                
    

In [99]:
items_for_user = {}

for user in user_pref:
    items_for_user[user] = []
    for item in items_to_predict:
        try:
            if len(set(movie_genres[item]).intersection(user_pref[user])) >= 1:
                items_for_user[user].append(item)
        except KeyError:
            #items_for_user[user].append(item)
            pass
        

In [105]:
model = LightFM(loss='logistic', no_components=10)

In [106]:
model.fit(train_coo, epochs=20)

In [83]:
recoms_genre = {}
num_to_recom = 5
for user in users_to_predict:
    #print(user)
    items_to_score = list(set(items_for_user[user]).difference(movie_user_hist[user]))
    if not items_to_score:
        items_to_score = list(items_to_predict)
    #print(len(items_to_score))
    predict = model.predict(user, items_to_score, num_threads=-1)
    top_recoms_id = sorted(range(len(predict)),
                           key=lambda i: predict[i])[-num_to_recom:]
    top_recoms_id.reverse()
    recoms_genre[user_decode[user]] = [item_decode[items_to_score[i]]
                                 for i in top_recoms_id]

metrics.mapk(list(recoms_genre.values()), list(correct_1.values()), 5)

KeyboardInterrupt: 

0.005921940928270043 > 100

In [107]:
recoms = {}
num_to_recom = 5
for user in users_to_predict:
    #print(user)
    items_to_score = list(set(items_to_predict).difference(movie_user_hist[user]))
    #print(len(items_to_score))
    predict = model.predict(user, items_to_score, num_threads=-1)
    top_recoms_id = sorted(range(len(predict)),
                           key=lambda i: predict[i])[-num_to_recom:]
    top_recoms_id.reverse()
    recoms[user_decode[user]] = [item_decode[items_to_score[i]]
                                 for i in top_recoms_id]

metrics.mapk(list(recoms.values()), list(correct_1.values()), 5)

0.007002109704641351

0.0067932489451476 idbm > 6

0.006348101265822786 > 100

In [26]:
movies_database = pd.read_csv('../data/raw/movies.csv')

In [83]:
N = 5

for ind, vals in enumerate(zip(recoms.items(), correct_1.values())):
    #print(vals)
    user = vals[0][0]
    pred, corr = set(vals[0][1]), set(vals[1])
    pred_genre = recoms_genre[user]
    error_rate = len(corr.intersection(pred)) / len(corr)
    print(error_rate)
    #if error_rate > 0:
    #    print(pred, corr)
    if (error_rate == 0) and (len(corr) >= 3):
        print(user)
        print('predict:')
        for movie in pred:
            film = movies_database[movies_database['id'] == movie]
            print(film['title'].values[0], film['genres'].values[0])
            
        print('predict with genre:')
        for movie in pred_genre:
            film = movies_database[movies_database['id'] == movie]
            try:
                print(film['title'].values[0], film['genres'].values[0])
            except:
                pass
            
        print('correct:')
        for movie in corr:
            film = movies_database[movies_database['id'] == movie]
            try:
                print(film['title'].values[0], film['genres'].values[0])
            except:
                pass
    print('--------------')

0.2
--------------
0.0
17025815393096229904
predict:
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
The Lego Ninjago Movie Kids
Hotel Transylvania Kids
Oggy and the Cockroaches: The Movie Kids
Hotel Transylvania 2 Kids
predict with genre:
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
The Nun Detective,Thriller,Horror
Tag Comedy
Captain Marvel Action,Adventure,Sci-Fi
Pixels Comedy,Action,Sci-Fi,Fantasy
correct:
Patrick Comedy
We Own the Night Thriller,Crime,Drama
Loving Pablo Crime,Drama
Moneyball Drama
--------------
0.0
8801367151077484144
predict:
Rampage Action,Adventure,Sci-Fi
Ready Player One Action,Adventure,Sci-Fi
Captive State Thriller,Sci-Fi,Drama
The Meg Action,Thriller,Horror,Sci-Fi
The Secret Life of Pets 2 Kids
predict with genre:
Rampage Action,Adventure,Sci-Fi
The Secret Life of Pets 2 Kids
Ready Player One Action,Adventure,Sci-Fi
The Meg Action,Thriller,Horror,Sci-Fi
Captive State Thriller,Sci-Fi,Drama
correct:
Deepwater Horizon Action,Thriller,Drama,

The Meg Action,Thriller,Horror,Sci-Fi
Ocean's Eight Comedy,Action,Thriller,Crime
Captive State Thriller,Sci-Fi,Drama
correct:
Penguins of Madagascar Kids
Monkey King Reloaded Kids
Escape Plan Action,Detective,Thriller,Sci-Fi,Crime
Mune: Guardian of the Moon Kids
Trolls Kids
The Smurfs Kids
Paddington Comedy,Adventure,Family
--------------
0.0
9846370997528762150
predict:
Rampage Action,Adventure,Sci-Fi
Ready Player One Action,Adventure,Sci-Fi
The Meg Action,Thriller,Horror,Sci-Fi
Ocean's Eight Comedy,Action,Thriller,Crime
The Secret Life of Pets 2 Kids
predict with genre:
Rampage Action,Adventure,Sci-Fi
Ready Player One Action,Adventure,Sci-Fi
The Meg Action,Thriller,Horror,Sci-Fi
Ocean's Eight Comedy,Action,Thriller,Crime
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
correct:
Penguins of Madagascar Kids
Abominable Kids
Maya the Bee Kids
Ozzy Kids
Kung Fu Panda 3 Kids
The Grinch Kids
The Star Kids
The Angry Birds Movie Kids
Free Birds Kids
Pride Comedy,Drama,History
Down a Da

Ocean's Eight Comedy,Action,Thriller,Crime
The Secret Life of Pets 2 Kids
predict with genre:
Rampage Action,Adventure,Sci-Fi
The Secret Life of Pets 2 Kids
Ready Player One Action,Adventure,Sci-Fi
Ocean's Eight Comedy,Action,Thriller,Crime
Captive State Thriller,Sci-Fi,Drama
correct:
The Peanut Butter Falcon Comedy,Adventure,Drama
DeadTectives Comedy,Horror
Ozzy Kids
Why Women Kill AMEDIATEKA
Paradise Hills Thriller,Sci-Fi,Fantasy
The Good Doctor AMEDIATEKA
Night Hunter Detective,Thriller,Crime
The Knick AMEDIATEKA
Killing Eve AMEDIATEKA
--------------
0.0
16433019193096643063
predict:
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
The Hustle Comedy,Crime
Long Shot Comedy,Melodrama
Hotel Transylvania Kids
Fast & Furious Presents: Hobbs & Shaw Action,Adventure
predict with genre:
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
Hotel Transylvania Kids
The Hustle Comedy,Crime
Long Shot Comedy,Melodrama
Fast & Furious Presents: Hobbs & Shaw Action,Adventure
correct:
Pengu

Cold Skin Thriller,Horror,Adventure,Sci-Fi
This Is the End Comedy,Sci-Fi
Demolition Comedy,Drama
Polina Adventure,Fantasy
Time Freak Comedy,Melodrama,Sci-Fi,Drama
Non-Stop Action,Detective,Thriller
Happy Death Day 2U Comedy,Detective,Horror,Sci-Fi
Charming Kids
Ballerina Kids
Sing Kids
Trouble Kids
The Forbidden Kingdom Action,Sci-Fi
--------------
0.0
724031452980426670
predict:
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
Rampage Action,Adventure,Sci-Fi
John Wick: Chapter 3 - Parabellum Action,Thriller,Crime
The Meg Action,Thriller,Horror,Sci-Fi
Ocean's Eight Comedy,Action,Thriller,Crime
predict with genre:
Rampage Action,Adventure,Sci-Fi
The Meg Action,Thriller,Horror,Sci-Fi
Ocean's Eight Comedy,Action,Thriller,Crime
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
John Wick: Chapter 3 - Parabellum Action,Thriller,Crime
correct:
Delicacy Comedy,Melodrama
100 Things Comedy
Preacher: 1 Season Detective,Adventure,Drama
Destination Wedding Comedy,Melodrama,Drama
------

Wildling Thriller,Horror,Drama,Fantasy
Brooklyn's Finest Thriller,Crime
Django Unchained Comedy,Adventure,Drama
--------------
0.0
162878539774941703
predict:
Robin Hood Action,Thriller,Adventure
Asterix: The Secret of the Magic Potion Kids
John Wick: Chapter 3 - Parabellum Action,Thriller,Crime
Hotel Transylvania Kids
Fast & Furious Presents: Hobbs & Shaw Action,Adventure
predict with genre:
Hotel Transylvania Kids
John Wick: Chapter 3 - Parabellum Action,Thriller,Crime
Robin Hood Action,Thriller,Adventure
Asterix: The Secret of the Magic Potion Kids
Fast & Furious Presents: Hobbs & Shaw Action,Adventure
correct:
Rock Dog Kids
Robinson Crusoe: The Wild Life Kids
Shrek 2 Kids
Frankie Drake Mysteries: Season 1 Detective,Crime,Drama
Ready Player One Action,Adventure,Sci-Fi
The Duchess Melodrama,Drama,History,Biography
Hellboy Action,Adventure,Fantasy
The Ash Lad: In the Hall of the Mountain King Adventure,Family
The Ash Lad: In Search of the Golden Castle Adventure,Fantasy
Shrek Kids
---

...

Robin Hood Action,Thriller,Adventure
John Wick: Chapter 3 - Parabellum Action,Thriller,Crime
Asterix: The Secret of the Magic Potion Kids
The Secret Life of Pets 2 Kids
predict with genre:
The Secret Life of Pets 2 Kids
John Wick: Chapter 3 - Parabellum Action,Thriller,Crime
Robin Hood Action,Thriller,Adventure
Asterix: The Secret of the Magic Potion Kids
The Hustle Comedy,Crime
correct:
Amundsen Adventure,Drama,History,Biography
Marvel's Agents of S.H.I.E.L.D.: 1 Season Action,Thriller,Adventure,Sci-Fi,Drama
Source Code Action,Thriller,Sci-Fi,Drama
Don't Breathe Thriller,Horror,Crime
Attila Marcel Comedy
The Aftermath Melodrama,Drama,War
Zero Dark Thirty Thriller,Drama
The Invisible Guest Detective,Thriller,Crime
Warcraft Action,Adventure,Fantasy
Prisoners Detective,Thriller,Crime,Drama
--------------
0.0
2926456635527226789
predict:
Long Shot Comedy,Melodrama
Asterix: The Secret of the Magic Potion Kids
Fast & Furious Presents: Hobbs & Shaw Action,Adventure
The Lego Ninjago Movie Kid

Hoodwinked Too! Hood VS. Evil Adventure
Green Book Comedy,Drama,Biography
--------------
0.0
2708342410182063429
predict:
The Hustle Comedy,Crime
Ready Player One Action,Adventure,Sci-Fi
Oggy and the Cockroaches: The Movie Kids
Asterix: The Secret of the Magic Potion Kids
Hotel Transylvania 2 Kids
predict with genre:
Ready Player One Action,Adventure,Sci-Fi
The Hustle Comedy,Crime
Asterix: The Secret of the Magic Potion Kids
Oggy and the Cockroaches: The Movie Kids
Hotel Transylvania 2 Kids
correct:
Sing Kids
Smurfs: The Lost Village Kids
Loving Pablo Crime,Drama
The Professor and the Madman Detective,Thriller,Drama,Biography
The Cold Light of Day Thriller
The Secret Life of Pets 2 Kids
--------------
0.0
7801229993835138316
predict:
The Hustle Comedy,Crime
Robin Hood Action,Thriller,Adventure
John Wick: Chapter 3 - Parabellum Action,Thriller,Crime
Asterix: The Secret of the Magic Potion Kids
Captive State Thriller,Sci-Fi,Drama
predict with genre:
Captive State Thriller,Sci-Fi,Drama
Jo

Revolver Action,Detective,Thriller,Crime,Drama
The Angry Birds Movie Kids
The 5th Wave Action,Thriller,Adventure,Sci-Fi
Smurfs: The Lost Village Kids
Kung Fu Panda 3 Kids
The Queen's Corgi Kids
Toni Erdmann Comedy,Drama
A Simple Favor Comedy,Detective,Thriller,Crime
Bates Motel: 4 Season Detective,Thriller,Horror,Drama
Elementary: Season 6 Detective,Crime,Drama
Spider-Man: Homecoming Action,Adventure,Sci-Fi
From the Land of the Moon Melodrama,Drama
Charming Kids
Killer Elite Action,Thriller,Crime
The Equalizer Action,Thriller,Crime
Ben-Hur Action,Adventure,Drama,History
Scary Movie Comedy
Planet 51 Adventure
The Darkest Hour Thriller,Sci-Fi
--------------
0.0
3524933757362153976
predict:
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
Rampage Action,Adventure,Sci-Fi
Hotel Transylvania Kids
Captive State Thriller,Sci-Fi,Drama
The Meg Action,Thriller,Horror,Sci-Fi
predict with genre:
Rampage Action,Adventure,Sci-Fi
The Meg Action,Thriller,Horror,Sci-Fi
Captive State Thriller,Sci-

Ready Player One Action,Adventure,Sci-Fi
Ocean's Eight Comedy,Action,Thriller,Crime
The Meg Action,Thriller,Horror,Sci-Fi
correct:
Stealth Action,Sci-Fi
We Own the Night Thriller,Crime,Drama
Surrogates Action,Thriller,Sci-Fi
Loving Pablo Crime,Drama
The Professor and the Madman Detective,Thriller,Drama,Biography
Battle: Los Angeles Action,Sci-Fi
S.W.A.T. Action,Thriller,Adventure,Crime
The Recruit Thriller
--------------
0.0
5689436685420108176
predict:
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
Rampage Action,Adventure,Sci-Fi
Hotel Transylvania Kids
The Meg Action,Thriller,Horror,Sci-Fi
The Secret Life of Pets 2 Kids
predict with genre:
Rampage Action,Adventure,Sci-Fi
The Secret Life of Pets 2 Kids
The Meg Action,Thriller,Horror,Sci-Fi
Mortal Engines Action,Thriller,Adventure,Sci-Fi,Fantasy
Hotel Transylvania Kids
correct:
The Secret Life of Pets Kids
World on Fire Drama,War
12 Monkeys: Season 3 Detective,Thriller,Adventure,Sci-Fi,Drama
Quantum of Solace Action,Thriller,A

KeyboardInterrupt: 

In [82]:
movie

11368

In [76]:
corr

{3607, 11368, 15252, 16366, 17227}

In [35]:
user_pref[user_encode[13233854463023521736]]

['Comedy', 'Melodrama', 'Drama', 'Thriller', 'Action']

In [29]:
movie

6028599184390783311

In [83]:
users_to_predict = [user_encode[x] for x in list(test_2['user_id'])]

In [84]:
recoms = {}
num_to_recom = 5
for user in users_to_predict:
    #print(user)
    items_to_score = list(items_to_predict.difference(user_hist[user]))
    predict = model.predict(user, items_to_score, num_threads=-1)
    top_recoms_id = sorted(range(len(predict)),
                           key=lambda i: predict[i])[-num_to_recom:]
    top_recoms_id.reverse()
    recoms[user_decode[user]] = [item_decode[items_to_score[i]]
                                 for i in top_recoms_id]

In [85]:
metrics.mapk(list(recoms.values()), list(correct_1.values()), 5)

0.006761603375527427

In [160]:
metrics.mapk(list(recoms.values()), list(correct_1.values()), 5)

0.007573839662447258